# Laboratory excercise 5

The notebook contains excercise connected to the auditory excerice 6. For any questions feel free to contact assistant: eda.jovicic@fer.hr

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd

# set seed for consistency
np.random.seed(2)

1. Load the cleaned dataset from the first exercise. If you haven't saved the dataset, rerun the exercise and save the final dataset.

In [2]:
cd = pd.read_csv('cleaned_data.csv')
cd

,EthnicGroup,ParentEduc,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,TransportMeans,WklyStudyHours,StudentId,NrSiblings,MathScore,ReadingScore,WritingScore,GenderE,LunchTypeE
0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,520645.0,3.0,71.0,71.0,74.0,1.0,1.0
1,2.0,5.0,1.0,1.0,2.0,1.0,2.0,0.0,303683.0,0.0,69.0,90.0,88.0,1.0,1.0
2,1.0,3.0,1.0,2.0,2.0,1.0,1.0,1.0,457351.0,4.0,87.0,93.0,91.0,1.0,1.0
3,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,812988.0,1.0,45.0,56.0,42.0,1.0,1.0
4,2.0,5.0,1.0,1.0,2.0,1.0,1.0,0.0,594678.0,0.0,76.0,78.0,75.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,3.0,2.0,1.0,2.0,2.0,0.0,1.0,0.0,611241.0,2.0,59.0,61.0,65.0,1.0,1.0
29113,4.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,299810.0,1.0,58.0,53.0,51.0,1.0,1.0
29114,2.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,941589.0,1.0,61.0,70.0,67.0,1.0,1.0
29115,3.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,867225.0,3.0,82.0,90.0,93.0,1.0,1.0


2. Using linear regression, model the MathScore using features: TestPrep, NrSiblings, ParentMaritalStatus and ReadingScore. 
Questions:

a) Using the summary, write down the formula with coefficients

In [3]:
formula = 'MathScore ~ TestPrep + NrSiblings + ParentMaritalStatus + ReadingScore'
model = smf.ols(formula=formula, data=cd)
results = model.fit()

print(results.summary())

intercept = results.params['Intercept']
coefficients = results.params.drop('Intercept')  # Exclude the intercept

X = cd[['TestPrep', 'NrSiblings', 'ParentMaritalStatus', 'ReadingScore']]
y = cd['MathScore']

formula = f' = {intercept:.2f} + '
formula += ' + '.join([f'{coeff:.2f} * {feature}' for coeff, feature in zip(coefficients, X.columns)])
print('Formula za MathScore', formula)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train).fit()

X_test = sm.add_constant(X_test)
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')


                            OLS Regression Results                            
Dep. Variable:              MathScore   R-squared:                       0.669
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                 1.474e+04
Date:                Sun, 07 Jan 2024   Prob (F-statistic):               0.00
Time:                        18:44:35   Log-Likelihood:            -1.0471e+05
No. Observations:               29117   AIC:                         2.094e+05
Df Residuals:                   29112   BIC:                         2.095e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               6.3552    

b) Are there any statisticaly important features?
Based on the "P>|t|" column in the regression results summary table, we can conclude that the significant predictors for the variable MathScore are the variables TestPrep and ReadingScore (threshold: p<0.05)

c) Let's interpret the data, if students do test preparation, how does that affect the outcome on the math exam? 
To arrive at this conclusion, we examine two columns. One of them is "P>|t|," analyzed in the previous task, from which we know that the TestPrep variable is statistically significant. The other column is "coef," based on which we infer that the same variable is listed as the first predictor, and a coefficient with a positive sign indicates an increase in the dependent variable (MathScore) and the independent variable (TestPrep) in the same direction.

d) Are students who practice sport better at math exams or worse? Using linear regression, model the MathScore using only one feature: PracticeSport. Don't include intercept. Using summary, decide which students do the best at math exams - those who never practice sport, sometimes practice sport or those who do it regularly?

In [35]:
X = cd['PracticeSport']
y = cd['MathScore']

X = sm.add_constant(X)  
model = sm.OLS(y, X).fit()

coef_practice_sport = model.params['PracticeSport']
p_value_practice_sport = model.pvalues['PracticeSport']

print(f"Coefficient for PracticeSport: {coef_practice_sport}")
print(f"P-value for PracticeSport: {p_value_practice_sport}")

if coef_practice_sport > 0 and p_value_practice_sport < 0.05:
    print("Students who practice sports tend to have higher math scores.")

    
    if coef_practice_sport > 0:
        print("Never Practice Sport: These students might have lower math scores compared to the other groups.")
    if coef_practice_sport > 0:
        print("Sometimes Practice Sport: These students might have intermediate math scores.")
    if coef_practice_sport > 0:
        print("Regularly Practice Sport: These students might have the highest math scores.")
else:
    print("The relationship between sports practice and math scores is not statistically significant.")

Coefficient for PracticeSport: 0.36994315008797257
P-value for PracticeSport: 0.0035320420152137343
Students who practice sports tend to have higher math scores.
Never Practice Sport: These students might have lower math scores compared to the other groups.
Sometimes Practice Sport: These students might have intermediate math scores.
Regularly Practice Sport: These students might have the highest math scores.
